In [1]:
import sys,os
sys.path.append('../')
from smodels.base import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.decomposition import decomposer
from smodels.base.physicsUnits import fb, GeV, TeV
from smodels.matching.theoryPrediction import theoryPredictionsFor as tpNew
from smodels.matching.theoryPredictionOld import theoryPredictionsFor as tpOld
from smodels.experiment.databaseObj import Database
from smodels.matching.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.base.smodelsLogging import setLogLevel
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
setLogLevel("error")

# Set the path to the database
database = Database('official')

ModuleNotFoundError: No module named 'smodels.matching.theoryPredictionNew'

### Load model

In [ ]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [ ]:
# Set main options for decomposition
sigmacut = 0.5*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

### Compute the theory predictions (old method)

In [ ]:
allPredictions = []
for expResult in database.expResultList:
    predictions = tpOld(expResult, topDict, combinedResults=False, marginalize=False)
    if not predictions:
        continue  # Skip if there are no constraints from this result
    # print('\n %s ' % expResult.globalInfo.id)
    for theoryPrediction in predictions:
        allPredictions.append(theoryPrediction)
        continue
        dataset = theoryPrediction.dataset
        datasetID = theoryPrediction.dataId()
        txnames = sorted([str(txname) for txname in theoryPrediction.txnames])
        print("------------------------")
        print("Dataset = ", datasetID)  # Analysis name
        print("TxNames = ", txnames)
        print("Theory Prediction = ", theoryPrediction.xsection)  # Signal cross section
        print("Condition Violation = ", theoryPrediction.conditions)  # Condition violation values

        # Get the corresponding upper limit:
        print("UL for theory prediction = ", theoryPrediction.upperLimit)

        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)
        # Compute likelihoods for EM-type results:
        if dataset.getType() == 'efficiencyMap':
            theoryPrediction.computeStatistics()
            print('L_BSM, L_SM, L_max = %1.3E, %1.3E, %1.3E' % (theoryPrediction.likelihood(),
                  theoryPrediction.lsm(), theoryPrediction.lmax()))
        if r > rmax:
            rmax = r
            bestResult = expResult.globalInfo.id
        allPredictions.append(theoryPrediction)

### Compute the theory predictions (new method)

In [ ]:
allPredictionsNew = tpNew(database,topDict, combinedResults=False, marginalize=False)

### Compare results

In [ ]:
print(len(allPredictions))
print(len(allPredictionsNew))

In [ ]:
allPredictions = sorted(allPredictions, key = lambda tp: tp.getRValue(),reverse=True)
allPredictionsNew = sorted(allPredictionsNew, key = lambda tp: tp.getRValue(),reverse=True)

In [ ]:
for itp,tp in enumerate(allPredictions):
    resOld = [tp.analysisId(),tp.dataId(),tp.dataType(),tp.getRValue()]
    tpNew = allPredictionsNew[itp]
    resNew = [tpNew.analysisId(),tpNew.dataId(),tpNew.dataType(),tpNew.getRValue()]
    # Compare analysis info
    if resOld[:-1] != resNew[:-1]:
        print(resOld)
        print(resNew)
        break
    # Compare R value
    if 2*abs(resOld[-1]-resNew[-1])/(resOld[-1]+resNew[-1]) > 0.01:
        print(resOld)
        print(resNew)
        break